# Data Export
1. Copies cocktails_db.pkl into api/src <i>(to be deployed with the next lambda)</i>
2. Uploads ingredients_db.pkl to algolia

# 1. Copies cocktails_db.pkl into api/src

In [59]:
import os
os.system('cp cocktails_db.pkl ../../src/')

0

# 2. Uploads ingredients_db.pkl to Algolia

In [60]:
import pickle
from algoliasearch.search_client import SearchClient
client = SearchClient.create('XDZGBI1FCU', '81aed0e97eb67f4d0c54548d18fc2fff')
index = client.init_index('ingredients')
index.clear_objects()


In [61]:
import boto3

icdf = pickle.load(open('ingredients_db.pkl', 'rb'))
print('{} total ingredients'.format(len(icdf)))

# Remove ingredients without an image in S3
bucket_name='cocktail-ingredient-images'
session = boto3.Session(profile_name='personal')
client = session.client('s3', region_name='us-west-2')
s3_images = [key['Key'].replace('.png', '') for key in client.list_objects(Bucket=bucket_name)['Contents']]
icdf = icdf.loc[icdf.objectID.isin(s3_images)]

# Remove blacklisted category_ids
blacklist = ['water', 'ice']
icdf = icdf.loc[~icdf.objectID.isin(blacklist)]

to_upload = icdf.to_dict(orient='records')

print('{} ingredients to upload'.format(len(to_upload)))

394 total ingredients
186 ingredients to upload


In [62]:
index.save_objects(to_upload)
index.set_settings({"searchableAttributes": ["name"]})